In [2]:
import numpy as np
import pandas as pd
from os import makedirs
from os.path import join, exists
import bct
import datetime

In [5]:
subjects = ['101', '102', '103', '104', '106', '107', '108', '110', '212', '213',
            '214', '215', '216', '217', '218', '219', '320', '321', '322', '323',
            '324', '325', '327', '328', '329', '330', '331', '332', '333', '334',
            '335', '336', '337', '338', '339', '340', '341', '342', '343', '344',
            '345', '346', '347', '348', '349', '350', '451', '452', '453', '455',
            '456', '457', '458', '459', '460', '462', '463', '464', '465', '467',
            '468', '469', '470', '502', '503', '571', '572', '573', '574', '575',
            '577', '578', '579', '580', '581', '582', '584', '585', '586', '587',
            '588', '589', '590', '591', '592', '593', '594', '595', '596', '597',
            '598', '604', '605', '606', '607', '608', '609', '610', '611', '612',
            '613', '614', '615', '616', '617', '618', '619', '620', '621', '622',
            '623', '624', '625', '626', '627', '628', '629', '630', '631', '633',
            '634']
#subjects = ['101', '102']
kappa_upper = 0.21
kappa_lower = 0.31

sink_dir = '/Users/katherine/Dropbox/Projects/physics-retrieval/data/output'

shen = '/home/kbott006/physics-retrieval/shen2015_2mm_268_parcellation.nii.gz'
craddock = '/home/kbott006/physics-retrieval/craddock2012_tcorr05_2level_270_2mm.nii.gz'
masks = ['shen2015', 'craddock2012']

tasks = {'reas': [{'conditions': ['Reasoning', 'Baseline']},
                  {'runs': [0,1]}],
         'retr': [{'conditions': ['Physics', 'General']},
                  {'runs': [0,1]}], 
         'fci': [{'conditions': ['Physics', 'NonPhysics']},
                  {'runs': [0,1,2]}]}

sessions = ['0','1']
sesh = ['pre', 'post']
conds = ['high-level', 'lower-level']

lab_notebook_dir = '/home/kbott006/lab_notebook/'
index = pd.MultiIndex.from_product([subjects, sessions, tasks, conds, masks], names=['subject', 'session', 'task', 'condition', 'mask'])
lab_notebook = pd.DataFrame(index=index, columns=['start', 'end', 'errors'])

index = pd.MultiIndex.from_product([subjects, sessions, tasks, conds, masks], names=['subject', 'session', 'task', 'condition', 'mask'])

shen_ntwks = pd.read_csv('/Users/katherine/Dropbox/Projects/physics-retrieval/shen_268_parcellation_networklabels.csv', index_col=0, header=0)

In [6]:
crad_dmn = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/dmn_craddock_rois.txt', 
                       delimiter='  ')

crad_rcen = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/rcen_craddock_rois.txt', 
                       delimiter='  ')

crad_lcen = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/lcen_craddock_rois.txt', 
                       delimiter='  ')

crad_sal = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/sn_craddock_rois.txt', 
                       delimiter='  ')

shen_dmn = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/dmn_shen_rois.txt', 
                       delimiter='  ')

shen_rcen = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/rcen_shen_rois.txt', 
                       delimiter='  ')

shen_lcen = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/lcen_shen_rois.txt', 
                       delimiter='  ')
shen_sal = np.genfromtxt('/Users/katherine/Dropbox/Projects/physics-retrieval/data/sn_shen_rois.txt', 
                       delimiter='  ')

In [7]:
crad_dmn = np.unique(crad_dmn)
crad_rcen = np.unique(crad_rcen)
crad_lcen = np.unique(crad_lcen)
crad_sal = np.unique(crad_sal)

shen_dmn = np.unique(shen_dmn)
shen_rcen = np.unique(shen_rcen)
shen_lcen = np.unique(shen_lcen)
shen_sal = np.unique(shen_sal)

In [8]:
shen_dmn_noverlap = list(set(shen_dmn) - set(shen_sal) - set(shen_lcen) - set(shen_rcen))
shen_cen_noverlap = list(set(shen_lcen.tolist() + shen_rcen.tolist()) - set(shen_sal) - set(shen_dmn))
shen_sal_noverlap = list(set(shen_sal) - set(shen_dmn) - set(shen_lcen) - set(shen_rcen))

In [9]:
crad_dmn_noverlap = list(set(crad_dmn) - set(crad_sal) - set(crad_lcen) - set(crad_rcen))
crad_cen_noverlap = list(set(crad_lcen.tolist() + crad_rcen.tolist()) - set(crad_sal) - set(crad_dmn))
crad_sal_noverlap = list(set(crad_sal) - set(crad_dmn) - set(crad_lcen) - set(crad_rcen))

In [10]:
indices = []

indices.append(shen_dmn_noverlap)
indices.append(shen_cen_noverlap)
indices.append(shen_sal_noverlap)

In [8]:
indices = []

indices.append(crad_dmn_noverlap)
indices.append(crad_cen_noverlap)
indices.append(crad_sal_noverlap)

In [9]:
dmn_crad_labels = []
for idx in crad_dmn_noverlap:
    dmn_crad_labels.append(str(int(idx)) + 'dmn')

cen_crad_labels = []
for idx in crad_cen_noverlap:
    cen_crad_labels.append(str(int(idx)) + 'cen')
    
sal_crad_labels = []
for idx in crad_sal_noverlap:
    sal_crad_labels.append(str(int(idx)) + 'sal')

In [10]:
tri_crad_labels = dmn_crad_labels + cen_crad_labels + sal_crad_labels

In [11]:
crad_labels = pd.Series(tri_crad_labels, index=np.arange(0, len(tri_crad_labels)))
crad_labels.to_csv(join(sink_dir, 'tripartite_regions_craddock2012.csv'))

In [12]:
dmn_shen_labels = []
for idx in shen_dmn_noverlap:
    dmn_shen_labels.append(str(int(idx)) + 'dmn')

cen_shen_labels = []
for idx in shen_cen_noverlap:
    cen_shen_labels.append(str(int(idx)) + 'cen')
    
sal_shen_labels = []
for idx in shen_sal_noverlap:
    sal_shen_labels.append(str(int(idx)) + 'sal')

In [13]:
tri_shen_labels = dmn_shen_labels + cen_shen_labels + sal_shen_labels

In [14]:
shen_labels = pd.Series(tri_shen_labels, index=np.arange(0, len(tri_shen_labels)))
shen_labels.to_csv(join(sink_dir, 'tripartite_regions_shen2015.csv'))

In [15]:
indices = pd.MultiIndex.from_product([subjects, sessions, tasks.keys(), conds])

In [16]:
df = pd.DataFrame(index=indices, dtype=np.float64)

In [18]:
for subject in subjects:
    for session in sessions:
        for task in tasks.keys():
            for i in np.arange(0, len(tasks[task][0]['conditions'])):
                conditions = tasks[task][0]['conditions']

                try:
                    lab_notebook.at[(subject, session, task, conds[i], 'shen2015'),'start'] = str(datetime.datetime.now())
                    corrmat = np.genfromtxt(join(sink_dir, 
                                                 '{0}-session-{1}_{2}-{3}_shen2015-corrmat.csv'.format(subject, 
                                                                                                       session, 
                                                                                                       task, 
                                                                                                       conditions[i])), 
                                            delimiter=' ')
                    thresh = bct.threshold_proportional(corrmat, 0.5, copy=True)
                    for idx1 in np.arange(0,len(tri_shen_labels)):
                        for idx2 in np.arange(0,len(tri_shen_labels)):
                            if idx1 != idx2:
                                df.at[(subject, session, task, conds[i]), '{0}-{1}'.format(tri_shen_labels[idx1], tri_shen_labels[idx2])] = thresh[idx1,idx2]
                except Exception as e:
                    #print(subject, session, task, conds[i], e)
                    pass
        df.to_csv(join(sink_dir, 'task-shen-triplenetwork.csv'))

In [19]:
df = pd.DataFrame(index=indices, dtype=np.float64)

In [20]:
for subject in subjects:
    for session in sessions:
        for task in tasks.keys():
            for i in np.arange(0, len(tasks[task][0]['conditions'])):
                conditions = tasks[task][0]['conditions']

                try:
                    corrmat = np.genfromtxt(join(sink_dir, 
                                                 '{0}-session-{1}_{2}-{3}_craddock2012-corrmat.csv'.format(subject, 
                                                                                                       session, 
                                                                                                       task, 
                                                                                                       conditions[i])), 
                                            delimiter=' ')
                    thresh = bct.threshold_proportional(corrmat, 0.5, copy=True)
                    for idx1 in np.arange(0,len(tri_crad_labels)):
                        for idx2 in np.arange(0,len(tri_crad_labels)):
                            if idx1 != idx2:
                                df.at[(subject, session, task, conds[i]), '{0}-{1}'.format(tri_crad_labels[idx1], tri_crad_labels[idx2])] = thresh[idx1,idx2]
                except Exception as e:
                    #print(subject, session, task, conds[i], e)
                    pass
        df.to_csv(join(sink_dir, 'task-crad-triplenetwork.csv'))

In [21]:
indices = pd.MultiIndex.from_product([subjects, sessions])

df_shen = pd.DataFrame(index=indices, dtype=np.float64)
df_crad = pd.DataFrame(index=indices, dtype=np.float64)

In [22]:
for subject in subjects:
    for session in sessions:
        try:
            corrmat = pd.read_csv(join(sink_dir, '{0}-session-{1}-rest_network_corrmat_shen2015.csv'.format(subject, session)), header=None, index_col=None)
            thresh = bct.threshold_proportional(corrmat.values, 0.5, copy=False)
            
            for idx1 in np.arange(0,len(tri_shen_labels)):
                for idx2 in np.arange(0,len(tri_shen_labels)):
                    if idx1 != idx2:
                        df_shen.at[(subject, session), '{0}-{1}'.format(tri_shen_labels[idx1], tri_shen_labels[idx2])] = thresh[idx1,idx2]
            
            corrmat = pd.read_csv(join(sink_dir, '{0}-session-{1}-rest_network_corrmat_craddock2012.csv'.format(subject, session)), header=None, index_col=None)
            thresh = bct.threshold_proportional(corrmat.values, 0.5, copy=False)
            for idx1 in np.arange(0,len(tri_crad_labels)):
                for idx2 in np.arange(0,len(tri_crad_labels)):
                    if idx1 != idx2:
                        df_crad.at[(subject, session), '{0}-{1}'.format(tri_crad_labels[idx1], tri_crad_labels[idx2])] = thresh[idx1,idx2]
        except Exception as e:
            print(subject, session,e)
            pass
df_shen.to_csv(join(sink_dir, 'rest-shen-triplenetwork.csv'))
df_crad.to_csv(join(sink_dir, 'rest-crad-triplenetwork.csv'))

213 0 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/213-session-0-rest_network_corrmat_shen2015.csv' does not exist
213 1 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/213-session-1-rest_network_corrmat_shen2015.csv' does not exist
322 0 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/322-session-0-rest_network_corrmat_shen2015.csv' does not exist
322 1 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/322-session-1-rest_network_corrmat_shen2015.csv' does not exist
329 0 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/329-session-0-rest_network_corrmat_shen2015.csv' does not exist
329 1 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/329-session-1-rest_network_corrmat_shen2015.csv' does not exist
332 0 File b'/Users/Katie/Dropbox/Projects/physics-retrieval/data/output/332-session-0-rest_network_corrmat_shen2015.csv' does not exist
332 1 File b'/Users/Katie/Dropbox/Project